In [1]:
from pyomo.environ import *
import pandapower as pp
import pandapower.networks as pn
import pandas as pd

ModuleNotFoundError: No module named 'pandapower'

In [153]:
path_cons = r'C:\Users\Dell\Documents\GitHub\PINNS_OPF\data\interim\asset_cons.csv'
path_ees = r'C:\Users\Dell\Documents\GitHub\PINNS_OPF\data\interim\asset_ees.csv'
path_gen = r'C:\Users\Dell\Documents\GitHub\PINNS_OPF\data\interim\asset_gen.csv'
path_pv = r'C:\Users\Dell\Documents\GitHub\PINNS_OPF\data\interim\asset_pv.csv'
path_bus = r'C:\Users\Dell\Documents\GitHub\PINNS_OPF\data\interim\bus5.csv'
path_line = r'C:\Users\Dell\Documents\GitHub\PINNS_OPF\data\interim\line5.csv'
path_time

In [154]:
# Inspect Netwrok
case5

This pandapower network includes the following parameter tables:
   - bus (5 elements)
   - load (3 elements)
   - sgen (1 element)
   - gen (3 elements)
   - ext_grid (1 element)
   - line (6 elements)
   - poly_cost (5 elements)
   - bus_geodata (5 elements)

In [155]:
case5.poly_cost

,element,et,cp0_eur,cp1_eur_per_mw,cp2_eur_per_mw2,cq0_eur,cq1_eur_per_mvar,cq2_eur_per_mvar2
0,0,gen,0.0,14.0,0.0,0.0,0.0,0.0
1,0,sgen,0.0,15.0,0.0,0.0,0.0,0.0
2,1,gen,0.0,30.0,0.0,0.0,0.0,0.0
3,0,ext_grid,0.0,40.0,0.0,0.0,0.0,0.0
4,2,gen,0.0,10.0,0.0,0.0,0.0,0.0


In [161]:
cost = {
    0: 15.0,
    1: 30.0,
    2: 10.0
}

In [157]:
df_gen = case5.gen
df_gen

,name,bus,p_mw,vm_pu,sn_mva,min_q_mvar,max_q_mvar,scaling,slack,in_service,slack_weight,type,controllable,max_p_mw,min_p_mw
0,None,0,40.00,1.0,NaN,-30.0,30.0,1.0,False,True,0.0,None,True,40.0,0.0
1,None,2,323.49,1.0,NaN,-390.0,390.0,1.0,False,True,0.0,None,True,520.0,0.0
2,None,4,466.51,1.0,NaN,-450.0,450.0,1.0,False,True,0.0,None,True,600.0,0.0


In [160]:
# Create the dictionaries
gen_bus = dict(zip(zip(case5.gen['bus'], case5.gen.index), case5.gen['max_p_mw']))
gen_bus

{(0, 0): 40.0, (2, 1): 520.0, (4, 2): 600.0}

In [110]:
# Extract characteristics for buses
B = list(case5.bus.name) # Name of buses
V_lb = list(case5.bus.min_vm_pu) # Volt lower bounds
V_ub = list(case5.bus.max_vm_pu) # Volt upper bounds

case5.bus

,name,vn_kv,type,zone,in_service,max_vm_pu,min_vm_pu
0,0,230.0,b,1.0,True,1.1,0.9
1,1,230.0,b,1.0,True,1.1,0.9
2,2,230.0,b,1.0,True,1.1,0.9
3,3,230.0,b,1.0,True,1.1,0.9
4,4,230.0,b,1.0,True,1.1,0.9


In [121]:
case5.line

,name,std_type,from_bus,to_bus,length_km,r_ohm_per_km,x_ohm_per_km,c_nf_per_km,g_us_per_km,max_i_ka,df,parallel,type,in_service,max_loading_percent
0,None,None,0,1,1.0,1.48649,14.8649,35.702054,0.0,1.004087,1.0,1,ol,True,100.0
1,None,None,0,3,1.0,1.60816,16.0816,32.994314,0.0,99999.000000,1.0,1,ol,True,100.0
2,None,None,0,4,1.0,0.33856,3.3856,156.748063,0.0,99999.000000,1.0,1,ol,True,100.0
3,None,None,1,2,1.0,0.57132,5.7132,92.865455,0.0,99999.000000,1.0,1,ol,True,100.0
4,None,None,2,3,1.0,1.57113,15.7113,33.796607,0.0,99999.000000,1.0,1,ol,True,100.0
5,None,None,3,4,1.0,1.57113,15.7113,33.796607,0.0,0.602452,1.0,1,ol,True,100.0


In [122]:
# Create the dictionaries
r_dict = dict(zip(zip(case5.line['from_bus'], case5.line['to_bus']), case5.line['r_ohm_per_km']))
x_dict = dict(zip(zip(case5.line['from_bus'], case5.line['to_bus']), case5.line['x_ohm_per_km']))
i_dict = dict(zip(zip(case5.line['from_bus'], case5.line['to_bus']), case5.line['max_i_ka']))

In [124]:
i_dict

{(0, 1): 1.00408742467761,
 (0, 3): 99999.0,
 (0, 4): 99999.0,
 (1, 2): 99999.0,
 (2, 3): 99999.0,
 (3, 4): 0.60245245480657}

In [162]:
# Consumption dict
p_cons = {
    (0,0) : -200,
    (0,1) : -50,
    (1,2) : -100,
    (1,3) : -85,
    (2,4) : -45,
    (3,5) : -55,
    (4,6) : -100,
    (4,7) : -98
}

In [130]:
model = ConcreteModel()


In [40]:
# Define variables
model.P_zb = Var(cons, within=Reals)  # active power for each parent node
#model.Q_zb = Var(cons, within=Reals)  # reactive power for each parent node
model.I_zb = Var(i_dict, within=NonNegativeReals)  # current for each branch
model.V_b = Var(B, within=NonNegativeReals)  # voltage for each bus

model.x_gen = Var(gen_bus, within=NonNegativeReals)
model.x_cons = Var(p_cons, within = NegativeReals)

# Define constraints
def power_balance_rule(model, b, t):
    return (model.P[(b, n), t] for n in N[b]) + sum(model.I[(b, c, t)]*(Rbc[(b, c)]**2 + Xbc[(b, c)]**2) for c in C[b]) \
           == model.P[(R[b], b), t] + sum(model.I[(c, b, t)]*(Rbc[(c, b)]**2 + Xbc[(c, b)]**2) for c in C[b])
model.power_balance = Constraint(B, T, rule=power_balance_rule)

def voltage_difference_rule(model, b, t):
    return model.V[(R[b], t)] - 2*Rbc[(R[b], b)]*model.P[(R[b], b), t] \
           - 2*Xbc[(R[b], b)]*model.Q[(R[b], b), t] - (Rbc[(R[b], b)]**2 + Xbc[(R[b], b)]**2)*model.I[(R[b], b, t)] \
           == model.V[(b, t)]
model.voltage_difference = Constraint(B, T, rule=voltage_difference_rule)

def branch_flow_rule(model, b, c, t):
    return model.V[(b, t)]*model.I[(b, c, t)] <= model.P[(b, c), t]**2 + model.Q[(b, c), t]**2
model.branch = Constraint(B,C, T, rule=branch_flow_rule)


ERROR: Rule failed when generating expression for Constraint power_balance
    with index (1, 0): KeyError: (1, 2)
ERROR: Constructing component 'power_balance' from data=None failed: KeyError:
    (1, 2)


KeyError: (1, 2)

In [1]:
from pyomo.environ import *

model = AbstractModel()

# Sets
model.B = Set() # set of buses
model.N = Set() # set of resources
model.T = Set() # set of time periods

# Parameters
model.p_max = Param(model.N) # maximum active power output of resource n
model.p_min = Param(model.N) # minimum active power output of resource n
model.q_max = Param(model.N) # maximum reactive power output of resource n
model.q_min = Param(model.N) # minimum reactive power output of resource n
model.r = Param(model.B,model.B) # resistance of line connecting buses b and c
model.x = Param(model.B,model.B) # reactance of line connecting buses b and c
model.pd = Param(model.B,model.T) # active power demand at bus b
model.qd = Param(model.B,model.T) # reactive power demand at bus b
model.v_min = Param(model.B) # minimum voltage at bus b
model.v_max = Param(model.B) # maximum voltage at bus b
model.i_max = Param(model.B, model.B) # maximum current flowing through line connecting buses b and c
model.cost = Param(model.N,model.T)


# Variables
model.x = Var(model.N, within=Reals) # decision variable for active output of resource n
model.q_n = Var(model.N, within=Reals) # reactive power output of resource n
model.p = Var(model.B, model.B, within=Reals) # active power flow of line connecting buses b and c
model.q = Var(model.B, model.B,within=Reals) # reactive power flow of line connecting buses b and c
model.v = Var(model.B,within=Reals) # voltage magnitude at bus b
model.i = Var(model.B, model.B, within=NonNegativeReals) # magnitude of current flowing through line connecting buses b and c


# Objective function
def obj_expression(model):
    return sum((model.cost[n, t] * model.x[n,t]*model.p_max[n] for n in model.N for t in model.T))

model.obj = Objective(rule=obj_expression, sense=minimize)

# Constraints
def power_balance_rule(model, b, t):
    return model.pd[b, t] + sum(model.p[n, t] for n in model.N if n in model.N_b[b]) - sum((model.p[c, t] + model.r[b, c] * model.i_sq[b, c, t]) for c in model.B if (b, c) in model.L) == 0

model.power_balance = Constraint(model.B, model.T, rule=power_balance_rule)

def voltage_balance_rule(model, b, t):
    return model.v[b, t] == sum(model.v[c, t] + model.r[b, c] * model.p[c, t] + model.x[b, c] * model.q[c, t] - model.i_sq[b, c, t] for c in model.B if (b, c) in model.L)

model.voltage_balance = Constraint(model.B, model.T, rule=voltage_balance_rule)

def current_limit_rule(model, b, c, t):
    return model.i_sq[b, c, t] <= model.i_max[b, c] * model.i_max[b, c]

model.current_limit = Constraint(model.B, model.B, model.T, rule=current_limit_rule)

def voltage_limit_rule(model, b, t):
    return inequality(model.v_min[b], model.v[b, t], model.v_max[b])

    'pyomo.core.base.param.IndexedParam'>) on block unknown with a new
    Component (type=<class 'pyomo.core.base.var.IndexedVar'>). This is usually
    indicative of a modelling error. To avoid this warning, use
    block.del_component() and block.add_component().


In [4]:
data = {
    # Sets
    'B': {1, 2, 3},
    'N': {1, 2},
    'T': {1, 2, 3, 4},

    # Parameters
    'p_max': {(1, 1): 100, (1, 2): 120, (1, 3): 130, (2, 1): 50, (2, 2): 60, (2, 3): 70},
    'p_min': {(1, 1): 0, (1, 2): 0, (1, 3): 0, (2, 1): 0, (2, 2): 0, (2, 3): 0},
    'q_max': {(1, 1): 50, (1, 2): 60, (1, 3): 70, (2, 1): 20, (2, 2): 30, (2, 3): 40},
    'q_min': {(1, 1): -50, (1, 2): -60, (1, 3): -70, (2, 1): -20, (2, 2): -30, (2, 3): -40},
    'r': {(1, 2): 0.1, (1, 3): 0.2, (2, 3): 0.15},
    'x': {(1, 2): 0.2, (1, 3): 0.3, (2, 3): 0.25},
    'pd': {(1, 1): 100, (1, 2): 110, (1, 3): 120, (2, 1): 50, (2, 2): 60, (2, 3): 70},
    'qd': {(1, 1): 20, (1, 2): 30, (1, 3): 40, (2, 1): 10, (2, 2): 15, (2, 3): 20},
    'v_min': {1: 0.9, 2: 0.9, 3: 0.9},
    'v_max': {1: 1.1, 2: 1.1, 3: 1.1},
    'i_max': {(1, 2): 100, (1, 3): 80, (2, 3): 120},
}

# Additional data for sets and parameters that were not defined in the data dictionary
N_b = {1: [1], 2: [2]} # resources connected to each bus
L = {(1, 2), (1, 3), (2, 3)} # lines connecting buses

# Create an instance of the model with the data
instance = model.create_instance(data=data, N_b=N_b, L=L)

solvername='glpk'
solverpath_folder= r'C:\Users\Dell\miniconda3' #does not need to be directly on c drive
solverpath_exe=r'C:\Users\Dell\miniconda3\pkgs\glpk-5.0-h8ffe710_0\Library\bin\glpsol' #does not need to be directly on c drive
sys.path.append(solverpath_folder)

solver=SolverFactory(solvername)

# Solve the optimization problem
solver = SolverFactory('glpk')
solver.solve(instance)

# Print the optimal objective value and variable values
print(f"Optimal objective value: {value(instance.obj)}")
print("Active power output:")
for n in instance.N:
    for t in instance.T:
        print

ERROR: Model.create_instance() passed the following unrecognized keyword
    arguments (which have been ignored):
        'N_b' 'L'
    solver 'glpk'


ApplicationError: No executable found for solver 'glpk'

In [8]:
instance = model.create_instance()
opt = SolverFactory('glpk')
opt.solve(instance) 

    solver 'glpk'


ApplicationError: No executable found for solver 'glpk'

In [3]:
import sys